# Commands

In [ ]:
# cd ~/asa2-clustering-logs ; jupython --no-tail --venv st-venv -o nohup-asa2-clustering-$HOSTNAME.out ~/notebooks/asa/eval/asa2-clustering.ipynb
# observe ~/asa2-clustering-logs/nohup-asa2-clustering-$HOSTNAME.out

# Init

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = False # isNotebook, False, True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Imports

In [ ]:
from newssource.asattribution.utils import *
from newssource.asattribution.asamin import *
from newssource.asa.asapreproc import *
from newssource.asa.models import *
from newssource.metrics.ndcg import *

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
if not isNotebook:
    matplotlib.use('Agg')
import random
import time
import pickle
import copy
from hashlib import md5

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.utils import multi_gpu_model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.utils import multi_gpu_model

In [ ]:
import statistics

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning.kerasutils import *
from machinelearning.kerasmodels import *

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning import kerasutils
from machinelearning.iterator import *
from machinelearning.metrics import *

In [ ]:
from keras.layers import concatenate, Input
from keras.models import Model
from keras.utils import plot_model

In [ ]:
import scipy

In [ ]:
from deepstyle.model import *
from machinelearning import tf2utils

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

In [ ]:
from newssource.dbert.utils import *

In [ ]:
from nlptools.topicmodeling import *

# Config

In [ ]:
logger = Logger(tmpDir("logs") + "/asa2-clustering-" + getHostname() + ".log")

In [ ]:
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
files = \
[
    'uset0-l50-dpl50-d18-bc10',
    'uset0-l50-dpl50-blogger.com',
    'uset0-l50-dpl50-breitbart.com',
    'uset0-l50-dpl50-businessinsider.com',
    'uset0-l50-dpl50-cnn.com',
    'uset0-l50-dpl50-guardian.co.uk',
    'uset0-l50-dpl50-livejournal.com',
    'uset0-l50-dpl50-nytimes.com',
    'uset0-l50-dpl50-theguardian.com',
    'uset0-l50-dpl50-washingtonpost.com',
    'uset1-l50-dpl50-blogger.com',
    'uset1-l50-dpl50-d18-bc10',
    'uset1-l50-dpl50-livejournal.com',
    'uset2-l50-dpl50-blogger.com',
    'uset2-l50-dpl50-d18-bc10',
    'uset2-l50-dpl50-livejournal.com',
    'uset3-l50-dpl50-blogger.com',
    'uset3-l50-dpl50-d18-bc10',
    'uset3-l50-dpl50-livejournal.com',
    'uset4-l50-dpl50-blogger.com',
    'uset4-l50-dpl50-d18-bc10',
    'uset4-l50-dpl50-livejournal.com',
]
files = [nosaveDir() + "/Data/Asa2/detok-usets/" + e + "/0.ndjson.bz2" for e in files]
bp(files, 5, logger)

In [ ]:
tipisNumbers = "92 95 90 05 87 01 81 84 85 82 93 62 58 88 57 03 56 86 59 07 06 83 63 61 02 60 94".split()
tipis = ['tipi' + e for e in sorted(tipisNumbers)]
bp(tipis, logger)

In [ ]:
# We get the uset for the current tipi:
association = associate(tipis, files[:len(tipis)])
bp(association, 5, logger)
if not "tipi" in getHostname() or isDocker() or isHostname("titanv"):
    file = association['tipi59']
else:
    if getHostname() not in association:
        log("Exiting because " + getHostname() + " is not in associations", logger)
        exit()
    file = association[getHostname()]
uset = file.split('/')[-2]

In [ ]:
# Parameters for results:
dataCol = "filtered_sentences" # filtered_sentences, sentences

In [ ]:
# Other parameters:
randomTestsAmount = 0
docLength = 1200

In [ ]:
# Logging:
log(getHostname() + " handles " + uset, logger)
log("dataCol: " + str(dataCol), logger)

In [ ]:
# Data:
hashs = []
docs = []
labels = []
flattenedDocs = []
flattenedAndLoweredDocs = []
truncatedDocs = []
truncatedAndLoweredDocs = []
detokSentences = []
detokDocs = []
for row in NDJson(file):
    detokSentences.append(row['filtered_detokenized_sentences'])
    detokDocs.append(row['filtered_detokenized'])
    sentences = row[dataCol]
    theHash = objectToHash(sentences)
    hashs.append(theHash)
    docs.append(sentences)
    labels.append(row["label"])
    flattenedDoc = flattenLists(sentences)
    flattenedDocs.append(flattenedDoc)
    flattenedAndLoweredDocs.append([e.lower() for e in flattenedDoc])
    truncatedDoc = flattenedDoc[:docLength]
    truncatedDocs.append(truncatedDoc)
    truncatedAndLoweredDocs.append([e.lower() for e in truncatedDoc])
bp(docs, logger)
tt.tic("Got documents")

In [ ]:
if isNotebook:
    bp(docs[0])
    bp(detokDocs[0])
    bp(detokSentences[0])
    bp(labels[0])
    bp(flattenedDocs[0])
    bp(flattenedAndLoweredDocs[0])
    bp(truncatedDocs[0])
    bp(truncatedAndLoweredDocs[0])

In [ ]:
(classes, indexLabels) = encodeMulticlassLabels(labels, encoding='index')
bp(indexLabels, logger)

# Results

In [ ]:
(user, password, host) = getOctodsMongoAuth()
results = SerializableDict('asa2-clustering', useMongodb=True, user=user, password=password, host=host, logger=logger)

In [ ]:
def getResultBase(model, metric, verbose=True, **kwargs):
    global logger
    global uset
    global dataCol
    global results
    currentResult = kwargs
    currentResult['model'] = model
    currentResult['metric'] = metric
    currentResult['uset'] = uset
    currentResult['dataCol'] = dataCol
    currentHash = objectToHash(currentResult)
    return (currentHash, currentResult)

In [ ]:
def addResult(model, metric, score, verbose=True, **kwargs):
    global logger
    global uset
    global dataCol
    global results
    (currentHash, currentResult) = getResultBase(model, metric, verbose=verbose, **kwargs)
    currentResult['score'] = score
    results[currentHash] = currentResult
    bp(currentResult, logger, 5, verbose=verbose)

# Results table

In [ ]:
if isNotebook:
    allResults = [e[1] for e in results.items()]
    # bp(allResults, logger)
    r = dict()
    for current in allResults:
        # dataCol = current['dataCol']
        uset = current['uset']
        model = current['model']
        metric = current['metric']
        score = current['score']
        if uset not in r: r[uset] = dict()
        if model not in r[uset]: r[uset][model] = dict()
        assert metric not in r[uset][model]
        r[uset][model][metric] = score
    bp(r, logger)

In [ ]:
if isNotebook:
    print("Models:")
    bp(set(r["uset0-l50-dpl50-d18-bc10"].keys()), 5)

In [ ]:
if isNotebook:
    ######## Uset ########
    # usetPattern = ".*"
    # usetPattern = uset
    # usetPattern = "uset0-l50-dpl50-d18-bc10"
    # usetPattern = ".*d18-bc10"
    usetPattern = ".*blogger.*"
    # usetPattern = ".*livejournal.*"
    # usetPattern = ".*washingtonpost.*"
    # usetPattern = ".*breitbart.*"
    # usetPattern = ".*businessinsider.*"
    # usetPattern = ".*cnn.*"
    # usetPattern = ".*guardian.*co.*uk.*"
    # usetPattern = ".*theguardian.*"
    # usetPattern = ".*nytimes.*"
    # usetPattern = ".*c50.*"
    ######## Model ########
    # modelPattern = ".*tfidf$"
    # modelPattern = ".*nmf.*"
    # modelPattern = ".*lda.*"
    # modelPattern = ".*tfidf.*50$"
    # modelPattern = ".*dbert.*-distilbert-mean"
    # modelPattern = ".*dbert.*"
    modelPattern = ".*dbert.*ep32.*-distilbert-mean"
    # modelPattern = "sna"
    # modelPattern = "lstm"
    # modelPattern = "bert"
    # modelPattern = "stylo"
    # modelPattern = ".*usent.*mean.*"
    # modelPattern = ".*usent.*full.*"
    # modelPattern = ".*infersent-1.*mean.*"
    # modelPattern = ".*infersent-1.*full.*"
    # modelPattern = ".*word2vec.*"
    # modelPattern = ".*doc2vec.*"
    # modelPattern = ".*sent2vec.*mean.*"
    # modelPattern = ".*sent2vec.*full.*"
    # modelPattern = ".*random.*"
    ######## Metric ########
    metricPattern = ".*"
    # metricPattern = "simrank"
    # metricPattern = "calhar"
    # metricPattern = "davb"
    ######## Filtering ########
    fr = dict()
    for uset, models in r.items():
        if re.match(usetPattern, uset):
            for model, metrs in models.items():
                if re.match(modelPattern, model):
                    for metric, score in metrs.items():
                        if re.match(metricPattern, metric):
                            if uset not in fr: fr[uset] = dict()
                            if model not in fr[uset]: fr[uset][model] = dict()
                            fr[uset][model][metric] = score
    ######## Print ########
    bp(fr, 5, logger)

In [ ]:
if isNotebook:
    models = set(flattenLists([list(v.keys()) for k, v in fr.items()]))
    scores = dict()
    for model in models:
        scores[model] = dict()
    for uset, models in fr.items():
        for model, metrs in models.items():
            for metric, score in metrs.items():
                if metric not in scores[model]: scores[model][metric] = []
                scores[model][metric].append(score)
    log("For these usets: " + str(set(fr.keys())), logger)
    for model, metrs in scores.items():
        for metric, currentScores in metrs.items():
            token = model + " (" + str(len(currentScores)) + ")" + " ==> " + metric
            log(token, logger)
            meanScore = sum(currentScores) / len(currentScores)
            meanScore = truncateFloat(meanScore, 3)
            log(meanScore, logger)
    # bp(scores, 5)

In [ ]:
# Entire line:
if isNotebook:
    ######## Uset ########
    # usetPattern = ".*"
    # usetPattern = ".*d18-bc10"
    # usetPattern = ".*blogger.*"
    usetPatterns = [".*livejournal.*", ".*washingtonpost.*", ".*breitbart.*", ".*businessinsider.*", ".*cnn.*", ".*guardian.*co.*uk.*", ".*theguardian.*", ".*nytimes.*"]
    text = ""
    for usetPattern in usetPatterns:
        ######## Model ########
        # modelPattern = ".*tfidf$"
        # modelPattern = ".*tfidf.*50$"
        # modelPattern = ".*dbert.*None.*-mean"
        modelPattern = ".*nmf.*"
        # modelPattern = ".*lda.*"
        # modelPattern = "sna"
        # modelPattern = "lstm"
        # modelPattern = "bert"
        # modelPattern = "stylo"
        # modelPattern = ".*usent.*mean.*"
        # modelPattern = ".*usent.*full.*"
        # modelPattern = ".*infersent-1.*mean.*"
        # modelPattern = ".*infersent-1.*full.*"
        # modelPattern = ".*word2vec.*"
        # modelPattern = ".*doc2vec.*"
        # modelPattern = ".*sent2vec.*mean.*"
        # modelPattern = ".*sent2vec.*full.*"
        # modelPattern = ".*random.*"
        ######## Metric ########
        # metricPattern = ".*"
        metricPattern = "simrank"
        # metricPattern = "calhar"
        # metricPattern = "davb"
        ######## Filtering ########
        fr = dict()
        for uset, models in r.items():
            if re.match(usetPattern, uset):
                for model, metrs in models.items():
                    if re.match(modelPattern, model):
                        for metric, score in metrs.items():
                            if re.match(metricPattern, metric):
                                if uset not in fr: fr[uset] = dict()
                                if model not in fr[uset]: fr[uset][model] = dict()
                                fr[uset][model][metric] = score
        ######## Print ########
        models = set(flattenLists([list(v.keys()) for k, v in fr.items()]))
        scores = dict()
        for model in models:
            scores[model] = dict()
        for uset, models in fr.items():
            for model, metrs in models.items():
                for metric, score in metrs.items():
                    if metric not in scores[model]: scores[model][metric] = []
                    scores[model][metric].append(score)
        for model, metrs in scores.items():
            for metric, currentScores in metrs.items():
                token = model + " (" + str(len(currentScores)) + ")" + " ==> " + metric
                meanScore = sum(currentScores) / len(currentScores)
                meanScore = truncateFloat(meanScore, 3)
                text += str(meanScore) + "\t"
    log(text, logger)

In [ ]:
# The best model is dbert-94bef_ep32-distilbert-mean

# LDA

In [ ]:
# We set vars:
data = ldaFeatures(flattenedDocs, logger=logger)
model = 'lda'
meta = {}

In [ ]:
data.shape

In [ ]:
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)
score = metrics.calinski_harabasz_score(data, indexLabels)
addResult(model, 'calhar', score, **meta)
score = metrics.davies_bouldin_score(data, indexLabels)
addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("LDA done")

# NMF

In [ ]:
# We set vars:
data = nmfFeatures(flattenedDocs, logger=logger)
model = 'nmf'
meta = {}

In [ ]:
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)
score = metrics.calinski_harabasz_score(data, indexLabels)
addResult(model, 'calhar', score, **meta)
score = metrics.davies_bouldin_score(data, indexLabels)
addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("NMF done")

# DBert

In [ ]:
models = ['94bef_ep32']
log("DBert models: " + str(models), logger)

In [ ]:
modelsPath = []
for model in models:
    if model is None:
        modelsPath.append(None)
    else:
        if isDir(nosaveDir() + '/dbert-train/' + model):
            modelsPath.append(sortedGlob(nosaveDir() + '/dbert-train/' + model + '/epochs/ep*')[-1])
        else:
            modelsPath.append(sortedGlob(nosaveDir() + '/dbert-tmp/' + model + '/epochs/ep*')[-1])
bp(modelsPath, 5, logger)

In [ ]:
maxLength = 512
layers = ['distilbert']
operations = ['mean']

In [ ]:
pbar = ProgressBar(len(models) * len(operations) * len(layers), logger=logger, printRatio=0.001, message='Computing DBert scores')
for i in range(len(models)):
    modelName, modelPath, epoch = None, None, None
    if models[i] is not None:
        modelName = str(models[i])
        modelPath = modelsPath[i]
        epoch = getAllNumbers(modelPath)[-1]
    dbertCache = SerializableDict("dbert-embeddings-" + str(modelName),
                                  nosaveDir() + '/dbert-cache',
                                  useMongodb=False, logger=logger,
                                  loadRetry=30, loadSleepMin=0.5, loadSleepMax=30,
                                 readIsAnAction=False,)
    assert len(dbertCache) > 0
    log("Starting to compute DBert scores for " + str(modelName), logger)
    for layer in layers:
        embeddings = []
        for doc in docs:
            (currentHash, cacheObject) = getDbertEmbeddingsHash(doc, layer, modelPath, maxLength)
            assert currentHash in dbertCache
            embeddings.append(dbertCache[currentHash]['embeddings'])
        for operation in operations:
            dbertData = []
            if operation == 'first':
                for emb in embeddings:
                    dbertData.append(emb[0])
            elif operation == 'mean':
                for emb in embeddings:
                    dbertData.append(np.mean(emb, axis=0))
            # We set vars:
            data = np.array(dbertData)
            model = 'dbert-' + str(modelName) + '-' + layer + '-' + operation
            meta = {'modelName': modelName, 'layer': layer, 'operation': operation, 'epoch': epoch}
            # We compute and store score:
            simMatrix = pairwiseCosineSimilarity(data)
            score = pairwiseSimNDCG(simMatrix, indexLabels)
            addResult(model, 'simrank', score, **meta)
            score = metrics.calinski_harabasz_score(data, indexLabels)
            addResult(model, 'calhar', score, **meta)
            score = metrics.davies_bouldin_score(data, indexLabels)
            addResult(model, 'davb', score, **meta)
            pbar.tic()

In [ ]:
tt.tic("DBert_done")

# Random

In [ ]:
dimensions = [10, 100]
rdAmount = 5

In [ ]:
rdReprs = []
for dim in dimensions:
    for i in range(rdAmount):
        rdReprs.append(np.random.rand(len(docs), dim))
bp(rdReprs, logger)

In [ ]:
simRankScores = []
calharScores = []
davbScores = []
for current in pb(rdReprs, message="Generating random scores", logger=logger):
    simMatrix = pairwiseCosineSimilarity(current)
    simRankScores.append(pairwiseSimNDCG(simMatrix, indexLabels))
    calharScores.append(metrics.calinski_harabasz_score(current, indexLabels))
    davbScores.append(metrics.davies_bouldin_score(current, indexLabels))
bp(simRankScores, logger)

In [ ]:
# We set vars:
model = "random"
meta = {'dimensions': dimensions, 'rdAmount': rdAmount}
# We compute and store score:
addResult(model, 'simrank', float(np.mean(simRankScores)), **meta)
addResult(model, 'calhar', float(np.mean(calharScores)), **meta)
addResult(model, 'davb', float(np.mean(davbScores)), **meta)

In [ ]:
tt.tic("Random done")

# SNA

In [ ]:
# 6ebdd3e05d4388c658ca2d5c53b0bc36 (filtered_sentences)
# 7fa22619af09e90724d2e3a8cf5db796 (sentences)
# 94cdfee38dc865ccc73f2eab5b1fb3a5 (LSTM filtered_sentences)
deepStyleModels = \
{
    "6ebdd3e05d4388c658ca2d5c53b0bc36": "sna",
    # "7fa22619af09e90724d2e3a8cf5db796": "sna-nf",
    "94cdfee38dc865ccc73f2eab5b1fb3a5": "lstm",
}

In [ ]:
for modelPattern, name in deepStyleModels.items():
    m = DeepStyle(nosaveDir() + "/asa2-train/" + modelPattern, logger=logger)
    embeddings = np.array([m.embed(doc) for doc in docs])
    # We set vars:
    data = embeddings
    model = name
    meta = {'pattern': modelPattern}
    # We compute and store score:
    simMatrix = pairwiseCosineSimilarity(data)
    score = pairwiseSimNDCG(simMatrix, indexLabels)
    addResult(model, 'simrank', score, **meta)
    score = metrics.calinski_harabasz_score(data, indexLabels)
    addResult(model, 'calhar', score, **meta)
    score = metrics.davies_bouldin_score(data, indexLabels)
    addResult(model, 'davb', score, **meta)

In [ ]:
# We get random scores:
randomScores = []
for i in range(randomTestsAmount):
    randomScores.append(pairwiseSimNDCG(simMatrix, shuffle(indexLabels)))
randomScore = np.mean(randomScores) if len(randomScores) > 0 else None
log("Shuffled labels mean SimRank: " + str(randomScore), logger)

In [ ]:
tt.tic("DeepStyle done")

# Doc2Vec

In [ ]:
def getD2v(path, loadModel=True):
    d2vConfig = None
    d2vModel = None
    if isFile(path + "/config.json"):
        d2vConfig = fromJsonFile(path + "/config.json")
    if loadModel:
        d2vModel = Doc2Vec.load(sortedGlob(path + "/*model*.d2v")[0])
    return (d2vModel, d2vConfig)

In [ ]:
d2vPath = nosaveDir() + "/d2v/d2vmodel-t-ds22.02g-s300-w3-n15-e15-lTrue-adFalse-7bb8a"
# d2vPath = nosaveDir() + "/d2v/d2vmodel-t-ds6.2g-s300-w3-n30-e15-lTrue-adFalse-4b2d0"

In [ ]:
# We get the pretrained d2v model:
log("Loading " + d2vPath, logger)
(d2vModel, d2vConfig) = getD2v(d2vPath)

In [ ]:
# We get d2v vectors:
d2vData = d2vTokenssToEmbeddings(truncatedAndLoweredDocs, d2vModel, logger=logger, verbose=False) # 4m 34.06s

In [ ]:
# We set vars:
data = d2vData
model = 'doc2vec'
meta = {'pattern': d2vPath}
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)
score = metrics.calinski_harabasz_score(data, indexLabels)
addResult(model, 'calhar', score, **meta)
score = metrics.davies_bouldin_score(data, indexLabels)
addResult(model, 'davb', score, **meta)

In [ ]:
# We get random scores:
randomScores = []
for i in range(randomTestsAmount):
    randomScores.append(pairwiseSimNDCG(simMatrix, shuffle(indexLabels)))
randomScore = np.mean(randomScores) if len(randomScores) > 0 else None
log("Shuffled labels mean SimRank: " + str(randomScore), logger)

In [ ]:
d2vModel = None

In [ ]:
tt.tic("Doc2Vec done")

# TFIDF

### Alone

In [ ]:
tfidfInstance = TFIDF(truncatedAndLoweredDocs, doLower=True, logger=logger)
tfidfData = tfidfInstance.getTFIDFMatrix()

In [ ]:
# We set vars:
data = tfidfData
model = 'tfidf'
meta = {'lowered': True}
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)

In [ ]:
tt.tic("TFIDF done")

### SVD

In [ ]:
nComponentss = [10, 50, 100, 500, 1000]

In [ ]:
for nComponents in nComponentss:
    svd = TruncatedSVD(n_components=nComponents, n_iter=7, random_state=42)
    svdTFIDFData = svd.fit_transform(tfidfData)
    # We set vars:
    data = svdTFIDFData
    model = 'tfidf+svd-' + str(nComponents)
    meta = {'lowered': True, 'nComponents': nComponents}
    # We compute and store score:
    simMatrix = pairwiseCosineSimilarity(data)
    score = pairwiseSimNDCG(simMatrix, indexLabels)
    addResult(model, 'simrank', score, **meta)
    score = metrics.calinski_harabasz_score(data, indexLabels)
    addResult(model, 'calhar', score, **meta)
    score = metrics.davies_bouldin_score(data, indexLabels)
    addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("TFIDF+SVD done")

# Stylo

In [ ]:
styloVectors = []
for text in pb(detokDocs, logger=logger):
    styloVectors.append(stylo(text, asNpArray=True))
styloVectors = np.array(styloVectors)
bp(styloVectors, logger)

In [ ]:
# We set vars:
data = styloVectors
model = 'stylo'
meta = {}
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)
score = metrics.calinski_harabasz_score(data, indexLabels)
addResult(model, 'calhar', score, **meta)
score = metrics.davies_bouldin_score(data, indexLabels)
addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("Stylo done")

# Word vectors

In [ ]:
wvPatterns = ["glove-840B", "glove-6B", "word2vec-googlenews", "fasttext-wiki-news-1M", "fasttext-crawl-2M"]

In [ ]:
for wvPattern in wvPatterns:
    try:
        emb = Embeddings(wvPattern, logger=logger, verbose=False)
        wordEmbeddings = emb.getVectors()
        lowered = emb.isLower()
        for operation in ['sum', 'mean']:
            wvData = tokensToEmbedding(truncatedDocs, wordEmbeddings, operation=operation,
                                       removeDuplicates=True, doLower=lowered, verbose=False)
            # We set vars:
            data = wvData
            model = "wordvectors-" + wvPattern + '-' + operation
            meta = {'wvPattern': wvPattern, 'operation': operation, 'lowered': lowered}
            # We compute and store score:
            simMatrix = pairwiseCosineSimilarity(data)
            score = pairwiseSimNDCG(simMatrix, indexLabels)
            addResult(model, 'simrank', score, **meta)
            score = metrics.calinski_harabasz_score(data, indexLabels)
            addResult(model, 'calhar', score, **meta)
            score = metrics.davies_bouldin_score(data, indexLabels)
            addResult(model, 'davb', score, **meta)
    except Exception as e:
        logException(e, logger)

In [ ]:
tt.tic("WordVectors done")

# Universal Sentence Encoder

In [ ]:
# We get all embeddgins from usent:
allHashes = set()
for doc in docs:
    docHash = objectToHash(doc)
    allHashes.add(docHash)
    for sentence in doc:
        theHash = objectToHash(sentence)
        allHashes.add(theHash)
bp(allHashes, logger)
usentEmbeddings = dict()
for usentEmbeddingsFile in pb(sortedGlob(nosaveDir() + "/usent/usentEmbedding*.pickle"),
                              printRatio=0.001, logger=logger, message="Getting Usent embeddings from all files"):
    current = deserialize(usentEmbeddingsFile)
    for theHash, value in current.items():
        if theHash in allHashes:
            usentEmbeddings[theHash] = value
bp(usentEmbeddings, logger)

In [ ]:
assert len(allHashes) == len(usentEmbeddings)

In [ ]:
operations = ["mean", "full"]

In [ ]:
for operation in operations:
    if operation == "full":
        data = []
        for doc in docs:
            theHash = objectToHash(doc)
            assert theHash in usentEmbeddings
            data.append(usentEmbeddings[theHash])
        data = np.array(data)
    elif operation == "mean":
        data = []
        for doc in docs:
            docEmbeddings = []
            for sentence in doc:
                theHash = objectToHash(sentence)
                assert theHash in usentEmbeddings
                docEmbeddings.append(usentEmbeddings[theHash])
            docEmbedding = np.mean(docEmbeddings, axis=0)
            data.append(docEmbedding)
        data = np.array(data)
    # We set vars:
    data = data
    model = "usent-" + operation
    meta = {'operation': operation}
    # We compute and store score:
    simMatrix = pairwiseCosineSimilarity(data)
    score = pairwiseSimNDCG(simMatrix, indexLabels)
    addResult(model, 'simrank', score, **meta)
    score = metrics.calinski_harabasz_score(data, indexLabels)
    addResult(model, 'calhar', score, **meta)
    score = metrics.davies_bouldin_score(data, indexLabels)
    addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("Usent done")

# InferSent

Best is `V=1` and `method="mean"`

In [ ]:
import nltk
nltk.download('punkt')
bash("git clone https://github.com/facebookresearch/InferSent infersent")
bash("touch infersent/__init__.py")
sys.path.append('.')

In [ ]:
if isDocker():
    bash("pip install torch")

In [ ]:
from infersent.models import InferSent
import torch

In [ ]:
Vs = [2, 1]
methods = ["full", "mean"] # ["full", "mean", "sum"]
pbar = ProgressBar(len(Vs) * len(methods), printRatio=0.001, logger=logger)
for V in Vs:
    MODEL_PATH = nosaveDir() + '/infersent/infersent%s.pkl' % V
    params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                    'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
    infersent = InferSent(params_model)
    infersent.load_state_dict(torch.load(MODEL_PATH))
    if V == 2:
        W2V_PATH = nosaveDir() + '/infersent/fastText/crawl-300d-2M.vec'
    else:
        W2V_PATH = nosaveDir() + '/infersent/GloVe/glove.840B.300d.txt'
    infersent.set_w2v_path(W2V_PATH)
    infersent.build_vocab(detokDocs, tokenize=True)
    for method in methods:
        if method == "full":
            detokDocsForInferSent = []
            for current in detokDocs:
                detokDocsForInferSent.append(current[:10000])
            isData = infersent.encode(detokDocsForInferSent, tokenize=True)
        elif method == "mean":
            isData = []
            for currentDetokSentences in detokSentences:
                embedding = np.mean(infersent.encode(currentDetokSentences, tokenize=True), axis=0)
                isData.append(embedding)
            isData = np.array(isData)
        # We set vars:
        data = isData
        model = "infersent-" + str(V) + '-' + method
        meta = {'V': V, 'operation': method}
        # We compute and store score:
        simMatrix = pairwiseCosineSimilarity(data)
        score = pairwiseSimNDCG(simMatrix, indexLabels)
        addResult(model, 'simrank', score, **meta)
        score = metrics.calinski_harabasz_score(data, indexLabels)
        addResult(model, 'calhar', score, **meta)
        score = metrics.davies_bouldin_score(data, indexLabels)
        addResult(model, 'davb', score, **meta)
        pbar.tic()

In [ ]:
tt.tic("InferSent done")

# Sentence-BERT

In [ ]:
bertCache = SerializableDict("newsid-bretcache", useMongodb=True, user=user, password=password, host=host, logger=logger)

In [ ]:
bertData = []
for doc in docs:
    theHash = objectToHash(doc)
    current = bertCache[theHash]
    assert current is not None
    bertData.append(current)
bertData = np.array(bertData)
bp(bertData, logger)

In [ ]:
# We set vars:
data = bertData
model = "bert"
meta = {}
# We compute and store score:
simMatrix = pairwiseCosineSimilarity(data)
score = pairwiseSimNDCG(simMatrix, indexLabels)
addResult(model, 'simrank', score, **meta)
score = metrics.calinski_harabasz_score(data, indexLabels)
addResult(model, 'calhar', score, **meta)
score = metrics.davies_bouldin_score(data, indexLabels)
addResult(model, 'davb', score, **meta)

In [ ]:
tt.tic("BERT done")

# Sent2Vec

In [ ]:
# pip install Cython ; git clone https://github.com/epfml/sent2vec.git ; cd ./sent2vec ; pip install .

In [ ]:
installSent2Vec()

In [ ]:
import sent2vec

In [ ]:
for modelName in ["wiki_unigrams.bin"]: # ["wiki_unigrams.bin", "wiki_bigrams.bin"]
    s2vModel = sent2vec.Sent2vecModel()
    s2vModel.load_model(nosaveDir() + '/sent2vec/' + modelName)
    for method in ["full", "mean"]: # ["full", "mean", "sum"]
        if method == "full":
            s2vData = s2vModel.embed_sentences(detokDocs)
        elif method == "mean":
            s2vData = []
            for currentDetokSentences in detokSentences:
                embedding = np.mean(s2vModel.embed_sentences(currentDetokSentences), axis=0)
                s2vData.append(embedding)
            s2vData = np.array(s2vData)
        # We set vars:
        data = s2vData
        model = "sent2vec-" + modelName + '-' + method
        meta = {'pattern': modelName, 'operation': method}
        # We compute and store score:
        simMatrix = pairwiseCosineSimilarity(data)
        score = pairwiseSimNDCG(simMatrix, indexLabels)
        addResult(model, 'simrank', score, **meta)
        score = metrics.calinski_harabasz_score(data, indexLabels)
        addResult(model, 'calhar', score, **meta)
        score = metrics.davies_bouldin_score(data, indexLabels)
        addResult(model, 'davb', score, **meta)
    try:
        s2vModel.release_shared_mem(nosaveDir() + '/sent2vec/' + modelName)
        s2vModel = None
    except Exception as e:
        logException(e, logger)

In [ ]:
tt.tic("Sent2Vec done")

# End

In [ ]:
tt.toc()